In [ ]:
!pip install transformers torch

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_id = "hyunseoki/ko-ref-llama2-7b"
peft_model_id = "./Llama2-KorQuAD2-dev-finetuning"   # Data Augmentation 적용 o

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_

In [2]:
prompt = "## Question: %s ## Answer: "

def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=False,
    )
    return tokenizer.decode(gened[0]).replace(q, "")

In [6]:
print(gen('파스칼레 소틸레의 스파이크 높이는 몇 cm인가?'))                #O 332cm
print(gen('안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?')) #O 사이프러스
print(gen('조영무의 가족관계는?'))                                      #X 할아버지 조순후 증 참찬문하부사, 할머니: 증 정부인 문천박씨, 아버지: 조세진, 어머니: 증 정경부인 영흥최씨, 부인: 신천김씨/재령강씨 아들: 조서, 조윤, 조전/조질, 조이
print(gen('조선시대 조영무와 왕자의 난에는 어떤 관계가 있을까?'))         #ㅁ <p><a>1400년</a>(정종 2) <a>도독</a>중외제군사<a>도진무</a>(都督中外諸軍事都鎭撫)로 병권을 장악하였으며, 그해 <a>회안대군</a> 방간이 사병을 일으켜 <a>제2차 왕자의 난</a>이 벌어지자 <a>회안대군</a> 일파를 진압하기 위해 군사를 이끌고 <a>이방원</a>을 지원하였다. 난이 진압된 뒤 좌명공신(佐命功臣) 1등에 녹훈되었다. 그 뒤 <a>삼군부</a><a>참판사</a>(三軍府參判事)가 되었다. <a>1401년</a>(태종 1년)초 <a>태종</a> 즉위 후 참찬문하부사에 임명되었다가 임시로 의정부사(議政府事)를 겸임하여 <a>의정부</a>일을 맡아보았다. </p>
print(gen('공법과 사법을 구별하려는 이유는 무엇인가?'))                   #X 강력한 중앙집권적 전제국가가 존재하던 로마시대, 근대에는 공사법 구별이 엄격하였다. 반면에 <a>개인주의</a>가 발달한 시대에는 사법의 영역이 확장되었다. 최근 <a>복지국가사상</a>이 발달하면서 공사법 구별이 엄격해지고 있다. 즉, 사적자치가 발달하면 공사법이 부정되고, 국가의 권력이 강력해지면 공사법의 구별이 엄격해진다. 이를 통해서, 공법을 사법에서 굳이 분리하려는 이유는, 국가와 개인이 소송할 때는, 개인과 개인이 소송할 때보다 국가에게 더 유리하게 판결을 하기 위해 공법이 사법에서 분리된다는 것을 알 수 있다. <a>영미법</a>은 1천년이 넘도록 공사법 구별이 없이 사법으로만 규율하였는데, 20세기에 들어서 공법이 등장하고 있다는 것은, 영미법에서 <a>개인주의</a>나 <a>사적자치</a>가 후퇴하고 국가주의가 강화되고 있다는 의미로도 해석될 수 있다.
print(gen('국제 사원수 학회는 몇 년도에 설립되었을까?'))                  #X 1899년
print(gen('체스터 니미츠의 초기 생애는 어떠했는가?'))                     #X 체스터 윌리엄 니미츠는 체스터 버나드 니미츠(Chester Bernhard Nimitz)와 애나 행크 니미츠(Anna Henke Nimitz) 부부 사이에서 태어났다. 현재 니미츠가 태어난 <a>텍사스주</a> <a>프레더릭스버그</a>의 집은 기념관이 되었다. 니미츠의 아버지는 니미츠가 태어나기 전에 돌아가셨다. 니미츠는 <a>독일</a> 상선의 선원이었던 할아버지 찰스 H 니미츠(Charles H. Nimitz)로부터 많은 영향을 받았는데, 니미츠의 할아버지는 니미츠에게 "바다는 - 인생과 마찬가지로 - 매우 엄격한 선생님(Stern taskmaster)이란다. 네가 바다와 함께 잘 해 나갈 수 있는 가장 좋은 방법은, 우선 네가 배울 수 있는 모든 것을 배운 다음, 일에 있어서 최선을 다하고, 그리고 걱정하지 않는 거란다. - 특히 네가 어쩔 수 없는 것들에 대해서 말이지" 라고 가르쳤다. </p><p>어렸을 적 니미츠는 <a>웨스트포인트</a>에 진학하여 육군 장교가 되기를 희망하였지만 가능한 자리가 없었다. 니미츠가 살던 지역의 <a>하원</a> 의원이었던 <a>제임스 L 슬레이든</a>(James L. Slayden)은 니미츠에게 해군에 한 자리가 있는데, 이를 가장 적합한 지원자에게 줄 것이라고 말하였다. 니미츠는 이것이 교육을 더 받을 수 있는 유일한 기회라 여겼고, 그 자리를 얻기 위해 여가 시간을 공부하면서 보냈다. <a>1901년</a> 니미츠는 <a>텍사스</a>의 12번째 하원 의원 선거구로서 <a>미국 해군사관학교</a>에 입학하였다. 이 후 <a>1905년</a> <a>1월</a> 144명 중 7등이라는 우수한 성적으로 졸업하였다. 그는 <a>제2차 세계 대전</a> 태평양 전역 내내 "아일랜드 호퍼(Island Hopper)" 로서 알려졌다. </p>
print(gen('일본이 진주만을 공습한 사건이 언제 일어났나요?'))              #O 1941년 12월 7일
print(gen('1907년 니미츠가 임관한 계급은?'))                            #X 소위
print(gen('스티비 원더가 눈 수술을 받았다는 루머는 사실인가?'))           #O 사실이 아니다
print(gen('마이 셰리 어무어가 리믹스되어 발매된 연도는?'))                #O 1969


c:\workspace\AI\KorQuAD\myvenv\Lib\site-packages\transformers\generation\configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s> 332cm</s>
<s> 사이프러스</s>
<s> 아버지는 조질(趙秩)이고, 어머니는 <a>조씨</a> 부인(趙氏 夫人)이다.</s>
<s> <p><a>1400년</a>(정종 2) <a>도독</a>중외제군사<a>도진무</a>(都督中外諸軍事都鎭撫)로 병권을 장악하였으며, 그해 <a>회안대군</a> 방간이 사병을 일으켜 <a>제2차 
<s> 공법은 공법만의 고유한 성질이 있어야 하기 때문이고, 사법은 사법만으로 유지되기 어렵기 때문이다.</s>
<s> 1976년</s>
<s> 니미츠는 체스터 버나드 니미츠(Chester Bernhard Nimitz)로 1885년 2월 25일에 텍사스주 크레이크스 크로스에서 태어났다. 그의 아버지는 미쉘 카를 니미츠(Michel Karl Nimitz)이며 어머
<s> 1941년 12월 7일</s>
<s> 사제</s>
<s> 사실이 아니다</s>
<s> 1969</s>


In [8]:
print(gen('혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까지 일까?')) #O 145
print(gen('최태원이 출감한 날짜는 언제일까?')) #O 2015년 8월 14일

c:\workspace\AI\KorQuAD\myvenv\Lib\site-packages\transformers\generation\configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s> 145</s>
<s> 2015년 8월 14</s>


# 후기
train 시 max_length 를 256 으로 설정해서 그런지 제대로 학습된 내용인듯 하여도 중간중간 끊겨있는 답변들이 보인다. 또한 Data Augmentation을 적용시키지 않고, 학습량이 적어서 그런지 정답률이 높진 않으나 어느정도의 학습이 진행되었다는것을 확인하기엔 무리가 없어 보인다.

그러나 학습이 어느정도 되었다 라는것일 뿐 약 4일간 학습시킨 결과로써는 불만족스러운것이 사실이며, 정제한 질의 데이터의 질이 좋지 않았던점, 학습 시 토큰의 길이 조절에 실패한 점, AutoTrain Library 를 사용함으로써 스스로가 여러가지 학습 방법 또는 설정을 제대로 익히지 못했다는 점에서 개선사항 또는 고칠점이 여럿 보인다.